In [ ]:
from src import SQL_Executor, SQL_Queries
import pandas as pd

%load_ext dotenv
%dotenv
executor = SQL_Executor()
query = SQL_Queries.Payment()

In [ ]:
df = executor.execute_query(query.define_dpd_aggregate())
df

In [ ]:
dpd90_df = df.groupby('transaction_id').agg({
    'dpd': 'max'
}).reset_index()

dpd90_df['dpd90'] = (dpd90_df['dpd'] >= 90).astype(int)
dpd90_df

In [ ]:
t_df = executor.execute_query(query.calculate_dpd90_trend())
result_df = t_df.merge(dpd90_df, on='transaction_id', how='left')

In [ ]:
labels = ['18-25', '26-35', '36-50', '51+']
bins = [18, 26, 36, 51, 100]
group_age = pd.cut(result_df['age'], bins=bins, labels=labels)
dpd90_rates_age = result_df.groupby(group_age, observed=True)['dpd90'].agg(
    dpd90_rate_age = lambda x: (x.sum() * 100).round(2)
).reset_index()

dpd90_rates_age

In [ ]:
bins = [0, 15000, 30000, 60000, float('inf')]
labels = ['0-15k', '15k-30k', '30k-60k', '60k+']

group_income = pd.cut(result_df['income'], bins=bins, labels=labels)
dpd90_rates_income = result_df.groupby(group_income, observed=True)['dpd90'].agg(
    dpd90_rate_income = lambda x: (x.mean() * 100).round(2)
).reset_index()

dpd90_rates_income